In [1]:
""" ------------------- Library Imports ------------------- """
import multiprocessing
import pandas as pd
import numpy as np
import time
import requests
import setuptools.dist
import json
from sklearn.metrics import roc_auc_score
from confluent_kafka import Producer
from river import stream
import matplotlib.pyplot as plt
import csv
import os
import mlflow
import subprocess
import concurrent.futures
from multiprocessing import Pool
from multiprocessing import Process
import threading
import sys
import gevent.monkey
gevent.monkey.patch_all()
import locust
from locust import HttpUser, task, between, events

C:\Users\adilm\AppData\Local\Temp\ipykernel_24984\3554635603.py:23: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (c:\\Users\\adilm\\OneDrive\\Documents\\large-scale-online-learning\\.venv\\Lib\\site-packages\\urllib3\\util\\ssl_.py)', 'urllib3.util (c:\\Users\\adilm\\OneDrive\\Documents\\large-scale-online-learning\\.venv\\Lib\\site-packages\\urllib3\\util\\__init__.py)']. 
  gevent.monkey.patch_all()
c:\Users\adilm\OneDrive\Documents\large-scale-online-learning\.venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
def get_pub_ip_addr_and_port(name):
    temp = !kubectl get services --all-namespaces
    node_port = ""
    for line in temp:
        if name in line:
            parts = line.split()
            if len(parts) >= 6:
                node_port = parts[5].split(':')[1].split('/')[0]  # Extract NodePort
    temp = !kubectl get pods -o wide | findstr /R {name}
    node_name = temp[0].split()[6]  # Node name is the 7th column (index 6)
    temp = !kubectl get nodes -o wide | findstr /R {node_name}
    pub_ip = temp[0].split()[6]  # Public IP is the
    return f"{pub_ip}:{node_port}"

def get_inf_ipaddress():
    # Get all pods in all namespaces in JSON format
    result = subprocess.run(
        ["kubectl", "get", "pods", "-A", "-o", "json"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        check=True
    )
    pods_info = json.loads(result.stdout)
    pod_ips = []
    for item in pods_info.get("items", []):
        metadata = item.get("metadata", {})
        name = metadata.get("name", "")
        status = item.get("status", {})
        pod_ip = status.get("podIP")
        if "api-inferencia" in name and pod_ip:
            pod_ips.append(pod_ip)
    return pod_ips

In [10]:
# Get the public IP address and port for the update and inference services
ip_address_update = "http://" + get_pub_ip_addr_and_port("upd")
ip_address_inference = "http://" + get_pub_ip_addr_and_port("inf")
# ip_address_update = "http://localhost/update-service"
# ip_address_inference = "http://localhost/inference-service"
print(f"Update Service IP: {ip_address_update}")
print(f"Inference Service IP: {ip_address_inference}")

Update Service IP: http://34.123.170.195:32002
Inference Service IP: http://34.133.6.92:32001


In [12]:
# List of experiments to run
experiments = [
    ["AGR_a-HT-old-1000", "AGR_a.csv"],
]
experiment = experiments[0]
# Get the MLflow service URI
mlflow_service_uri = "http://mlflow:5000"
kafka_service_uri = "kafka-service.kafka:9092"
inf_ipaddress = get_inf_ipaddress()
inf_load_urls = ["http://" + ip + ":5001/load" for ip in inf_ipaddress]
model_update_rate = experiment[0].split("-")[-1]  # Extracting the rate from the experiment name 

# FOR INFERENCE KAFKA
url_kafka = f'{ip_address_inference}/kafka-uri'
r_kafka = requests.post(url_kafka, json=(kafka_service_uri))
print(f"Kafka URI INFERENCE configured with response: {r_kafka.status_code}")

# FOR INFERENCE MLFLOW
url_kafka = f'{ip_address_inference}/mlflow-uri'
r_kafka = requests.post(url_kafka, json=(mlflow_service_uri))
print(f"MLFLOW URI INFERENCE configured with response: {r_kafka.status_code}")

update_to_pods = f"{ip_address_update}/pods"
r_inf_pods = requests.post(update_to_pods, json=inf_load_urls)
print(f"Pods loaded with response: {r_inf_pods.status_code}")

# # Reset Counter
url_count = f'{ip_address_update}/count'
r_count = requests.post(url_count, json=(0))
print(f"Counter reset with response: {r_count.status_code}")

# # Configure Rate
url_rate = f'{ip_address_update}/rate'
r_rate = requests.post(url_rate, json=(model_update_rate))
print(f"Rate configured with response: {r_rate.status_code}")

url_mlflow = f'{ip_address_update}/mlflow-uri'
r_mlflow = requests.post(url_mlflow, json=(mlflow_service_uri))
print(f"MLflow URI configured with response: {r_mlflow.status_code}")

url_kafka = f'{ip_address_update}/kafka-uri'
r_kafka = requests.post(url_kafka, json=(kafka_service_uri))
print(f"Kafka URI configured with response: {r_kafka.status_code}")

url_load = f'{ip_address_update}/load'
exp_name = experiment[0]  # Use the first experiment name
r_load = requests.post(url_load, json=(exp_name))
print(f"Experiment {exp_name} loaded with response: {r_load.status_code}")


Kafka URI INFERENCE configured with response: 200
MLFLOW URI INFERENCE configured with response: 200
Pods loaded with response: 200
Counter reset with response: 200
Rate configured with response: 200
MLflow URI configured with response: 200
Kafka URI configured with response: 200
Experiment AGR_a-HT-old-1000 loaded with response: 200


In [ ]:
# [SKIP] Simple testing of the inference service
df = pd.read_csv(r'C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\Datasets\csv\AGR_a_redu.csv')
count = 0
for idx, row in df.iterrows():
    # Convert row to list and prepare data
    data = row.tolist()
    
    # Send data to your prediction endpoint
    response = requests.post(ip_address_inference + "/predict", json=data)
    
    if response.status_code == 200:
        prediction_result = response.json()
        print(f"Row {idx}: Prediction = {prediction_result}")
    else:
        print(f"Row {idx}: Error - {response.status_code}")
    count += 1
    if count >= 5000:
        break

Row 0: Prediction = 0.9116829779381651
Row 1: Prediction = 0.844215789708802
Row 2: Prediction = 0.9069676946280463
Row 3: Prediction = 0.8919935793645656
Row 4: Prediction = 0.8751481819498957
Row 5: Prediction = 0.02326669978609882
Row 6: Prediction = 0.0
Row 7: Prediction = 0.0
Row 8: Prediction = 0.0
Row 9: Prediction = 0.8785242313372434
Row 10: Prediction = 0.0
Row 11: Prediction = 0.859350839099818
Row 12: Prediction = 0.7693671195252425
Row 13: Prediction = 0.7270379634810006
Row 14: Prediction = 0.02326669978609882
Row 15: Prediction = 0.8729028187848062
Row 16: Prediction = 0.0
Row 17: Prediction = 0.4308895781054581
Row 18: Prediction = 0.835069130683878
Row 19: Prediction = 0.11524277165979788
Row 20: Prediction = 0.7440207977492016
Row 21: Prediction = 0.02326669978609882
Row 22: Prediction = 0.02326669978609882
Row 23: Prediction = 0.0
Row 24: Prediction = 0.0
Row 25: Prediction = 0.8997573878890222
Row 26: Prediction = 0.02326669978609882
Row 27: Prediction = 0.023266699

Traceback (most recent call last):
  File "c:\Users\adilm\OneDrive\Documents\large-scale-online-learning\.venv\Lib\site-packages\gevent\_ffi\loop.py", line 270, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
    
KeyboardInterrupt
2025-08-09T09:21:45Z


KeyboardInterrupt: 

c:\Users\adilm\OneDrive\Documents\large-scale-online-learning\.venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [ ]:
# FOR LOCUST
# locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py
# locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py --users 1 --spawn-rate 1 --headless --csv=result_testing
# [BACKGROUND RUN]
# nohup locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py --users 1 --spawn-rate 1 --headless --csv=result_testing > locust.log 2>&1 &

import re

substitutions = {
    # C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\Serialization_Datasets\testing.ipynb
    # C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\Datasets\real_usage
    # go to real_usage using relative path
    "dataset_dir": "Datasets/real_usage/",
    "file_name": "AGR_a_real_test.csv",
    # "host1": "http://localhost/inference-service"  
    "host1": ip_address_inference
}

for var, value in substitutions.items():
    # For string values, ensure quotes in the replacement
    replacement = f'{var} = "{value}"'
    # Use sed to replace the line in locust-test.py
    file_path = 'locust-testing.py'

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    for var, value in substitutions.items():
        # Replace lines like: var = "old_value"
        content = re.sub(
            rf'^{var}\s*=.*$', 
            f'{var} = "{value}"', 
            content, 
            flags=re.MULTILINE
        )
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

In [16]:
def copy_file_to_docker(file_path, docker_path):
    """Copy a file from local to GCP VM and then into the Docker container."""
    filename = file_path.split("/")[-1] if "/" in file_path else file_path.split("\\")[-1]
    # Copy file to VM
    !gcloud compute scp --zone=us-central1-a {file_path} load-testing-instance:/home/adilm
    # Copy file from VM to Docker container
    copy_command = f"docker cp /home/adilm/{filename} load-testing-container:{docker_path}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{copy_command}"

def run_docker_command(inner_command):
    """
    Run a command inside load-testing-container on the GCP VM via SSH using docker exec.
    """
    docker_exec_template = "docker exec load-testing-container bash -c '{cmd}'"
    docker_command = docker_exec_template.format(cmd=inner_command)
    ssh_command = f"gcloud compute ssh load-testing-instance --zone=us-central1-a --command=\"{docker_command}\""
    !{ssh_command}

def copy_dir_to_docker(dir_path, docker_path):
    """Copy a directory from local to GCP VM and then into the Docker container."""
    dir_name = dir_path.split("/")[-1] if "/" in dir_path else dir_path.split("\\")[-1]
    # Clean up the VM directory 
    rm_command = f"rm -rf /home/adilm/{dir_name}"  # Clean up the VM directory after copying
    docker_rm_command = f"docker exec load-testing-container rm -rf {docker_path}{dir_name}"  # Clean up the Docker directory
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{rm_command}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{docker_rm_command}"
    # Copy directory to VM
    print(f"gcloud compute scp --zone=us-central1-a --recurse {dir_path} load-testing-instance:/home/adilm/")
    !gcloud compute scp --zone=us-central1-a --recurse {dir_path} load-testing-instance:/home/adilm/
    # Copy directory from VM to Docker container
    copy_command = f"docker cp /home/adilm/{dir_name} load-testing-container:{docker_path}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{copy_command}"

In [ ]:
docker_command = "docker run -d --name load-testing-container auliadil/load-testing-rodrigues:v1 tail -f /dev/null"
gcloud_template = "gcloud compute ssh standalone-load-tester --zone=us-central1-a"
!{gcloud_template} --command="{docker_command}"

In [ ]:
copy_file_to_docker("../Serialization_Datasets/locust-testing.py", "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/")
copy_file_to_docker("../Serialization_Datasets/check-kafka-lag.py", "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/")
copy_dir_to_docker('C:/Users/adilm/AppData/Roaming/gcloud', "/root/.config/gcloud")

In [ ]:

projects = !gcloud config get-value project
set_project_command = f"gcloud config set project {projects[0]}"
get_kubectl_command = "gcloud container clusters get-credentials two-node-cluster --zone us-central1-a"
# run_docker_command(set_project_command)
# run_docker_command(get_kubectl_command)
run_docker_command("kubectl get pods")

In [ ]:
locust_path = "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/locust-testing.py"
locust_command = f"locust -f {locust_path} --users 1 --spawn-rate 1 --headless --csv=result_testing"
run_docker_command(f"cd /app/large-scale-online-learning/ && source ../.python-venv/bin/activate && nohup {locust_command} > locust.log 2>&1 &")
run_docker_command("cd /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/ && nohup python3 check-kafka-lag.py > kafka_lag.log 2>&1 &")

In [ ]:
# check whether the script is running
run_docker_command("ps aux | grep check-kafka-lag.py")
run_docker_command("ps aux | grep locust-testing.py")

In [ ]:
# kill the process if it is running
# run_docker_command("pkill -f check-kafka-lag.py")
# run_docker_command("pkill -f locust-testing.py")